In [ ]:
!pip install openai-whisper transformers torch


In [ ]:
import whisper, requests, json
from transformers import pipeline

whisper_model = whisper.load_model("base")

ner = pipeline(
    "ner",
    model="samrawal/bert-base-uncased_clinical-ner",
    aggregation_strategy="simple"
)

audio = "recording.wav"
text = whisper_model.transcribe(audio)["text"]

entities = ner(text)
symptoms = [e["word"] for e in entities if e["entity_group"] == "problem"]

risk = 0.8 if len(symptoms) >= 2 else 0.4

requests.post(
    "https://ivr-backend-te24.onrender.com/ivr/update",
    data={
        "patient_id": 1,
        "transcript": text,
        "symptoms": json.dumps(symptoms),
        "followups": json.dumps(["Any chest pain at rest?"]),
        "risk": risk
    }
)
